# Quick Start

In this quick start we create an extension of the Grid object, generate a random instance of it and perform power flow calculations and modifications.


## Setting up a grid extension

This shows how to add extra values to a `Grid` object. If these are present in the PGM output they will be updated after a power flow calculation.


In [1]:
from dataclasses import dataclass

import numpy as np
from numpy.typing import NDArray

from power_grid_model_ds import GraphContainer, Grid
from power_grid_model_ds.arrays import LineArray, NodeArray


class ExtendedNodeArray(NodeArray):
    """Extends the node array with an output value"""

    _defaults = {"u": 0}

    u: NDArray[np.float64]


class ExtendedLineArray(LineArray):
    """Extends the line array with an output value"""

    _defaults = {"i_from": 0}

    i_from: NDArray[np.float64]


@dataclass
class ExtendedGrid(Grid):
    """
    This is my own grid to extend.
    """

    node: ExtendedNodeArray
    line: ExtendedLineArray
    graphs: GraphContainer

## Grid Generation

The `RadialGridGenerator` can be used to create a randomised grid of the preferred size


In [2]:
from power_grid_model_ds.generators import RadialGridGenerator

grid_generator = RadialGridGenerator(grid_class=ExtendedGrid, nr_nodes=5, nr_sources=1, nr_nops=0)
grid = grid_generator.run(seed=0)

## Feeder IDs

To analyse network structure from the arrays, you can use feeder ids which identify how the network is connected. All structures connected to a substation node through the same feeding branch get the same feeder_branch_id.


In [ ]:
grid.set_feeder_ids()
print(grid.node)
print(grid.line)

## Performing power flow calculations

Using the `PowerGridModelInterface` the `Grid` data can be provided to the calculation engine. Using `update_grid` values can be transferred to the `Grid` object.


In [ ]:
from power_grid_model_ds import PowerGridModelInterface

core_interface = PowerGridModelInterface(grid=grid)

core_interface.create_input_from_grid()
core_interface.calculate_power_flow()
core_interface.update_grid()

print(grid.node)
print(grid.line)

## Modifying the Grid

The Grid object can be changed by adding objects or changing normally open points (NOPs).


In [5]:
new_line = ExtendedLineArray(
    from_node=[3],
    to_node=[1],
    r1=[0.6],
    x1=[0.2],
    i_n=[100],
    tan1=[0.0],
    from_status=[1],
    to_status=[1],
    c1=[0.0],
)

grid.add_branch(branch=new_line)

old_line = grid.line.get(19)
grid.make_inactive(branch=old_line)

This will give new outputs of the previous functions


In [6]:
core_interface = PowerGridModelInterface(grid=grid)
core_interface.create_input_from_grid()
core_interface.calculate_power_flow()
core_interface.update_grid()

grid.set_feeder_ids()

In [ ]:
print(grid.node)
print(grid.line)

## Analyzing Grid structure

The `Grid` also contains a graph representation which can be used for analyzing structure. Such as finding the shortest path between two nodes.


In [ ]:
path, length = grid.graphs.active_graph.get_shortest_path(1, 4)
print(f"Shortest path: {path}, Length: {length}")